In [14]:
import sys
import os
sys.path.append(os.getcwd() + "/..")
from ws.scraper import Scraper
from selenium import webdriver
from ws.constants import *
import json
import pandas as pd
import numpy as np

In [2]:
LEAGUES

{'Brasileirao': {'countryid': 31, 'leagueid': 95},
 'Bundesliga': {'countryid': 81, 'leagueid': 3},
 'Chinese Super League': {'countryid': 45, 'leagueid': 162},
 'Eredivisie': {'countryid': 155, 'leagueid': 13},
 'La Liga': {'countryid': 206, 'leagueid': 4},
 'Liga NOS': {'countryid': 177, 'leagueid': 21},
 'Ligue 1': {'countryid': 74, 'leagueid': 22},
 'Premier League': {'countryid': 252, 'leagueid': 2},
 'Primera Argentina': {'countryid': 11, 'leagueid': 68},
 'Russian Premier League': {'countryid': 182, 'leagueid': 77},
 'Serie A': {'countryid': 108, 'leagueid': 5},
 'Super Lig': {'countryid': 225, 'leagueid': 17},
 'UEFA Champions League': {'countryid': 250, 'leagueid': 12},
 'UEFA Europa League': {'countryid': 250, 'leagueid': 30}}

In [3]:


league = "premier_league"
season = "2017-2018"
path = JSONPATH+league+"/"+season
files = list(map(lambda x: path+"/"+x,os.listdir(path)))

In [4]:
def jsonList(league,season,jsonpath=JSONPATH):
    
    leaguename = league.lower().replace(" ","_")
    path = jsonpath+leaguename+"/"+season
    files = filter(lambda x: "json" in x, list(map(lambda x: path+"/"+x,os.listdir(path))))
    
    return list(files)

In [5]:
jsonList("Premier League","2017-2018")

['../data/json/premier_league/2017-2018/1190174.json',
 '../data/json/premier_league/2017-2018/1190175.json',
 '../data/json/premier_league/2017-2018/1190176.json',
 '../data/json/premier_league/2017-2018/1190177.json',
 '../data/json/premier_league/2017-2018/1190178.json',
 '../data/json/premier_league/2017-2018/1190179.json',
 '../data/json/premier_league/2017-2018/1190180.json',
 '../data/json/premier_league/2017-2018/1190181.json',
 '../data/json/premier_league/2017-2018/1190182.json',
 '../data/json/premier_league/2017-2018/1190183.json',
 '../data/json/premier_league/2017-2018/1190184.json',
 '../data/json/premier_league/2017-2018/1190185.json',
 '../data/json/premier_league/2017-2018/1190186.json',
 '../data/json/premier_league/2017-2018/1190187.json',
 '../data/json/premier_league/2017-2018/1190188.json',
 '../data/json/premier_league/2017-2018/1190189.json',
 '../data/json/premier_league/2017-2018/1190190.json',
 '../data/json/premier_league/2017-2018/1190191.json',
 '../data/

In [6]:
class LeagueFolder():
    
    jsonpath = JSONPATH
    
    def __init__(self,league):
        
        self.league = league
        self.leaguename = self.league.lower().replace(" ","_")
        self.leaguepath = self.jsonpath+self.leaguename+"/"
        self.seasons = os.listdir(self.leaguepath)
        self.matcheslists = {s:list(map(lambda x: x.split(".")[0],os.listdir(self.leaguepath+s))) for s in self.seasons}
                
    def matchesList(self,season=None):
        
        return self.matcheslists[season] if season != None else self.matcheslists
    
    def numberMatches(self):
        
        return pd.Series({k:len(v) for k,v in self.matcheslists.items()},name="numberofmatches")

In [7]:
a = LeagueFolder("Premier League")
a.numberMatches()
a.matchesList("2017-2018")

['1190174',
 '1190175',
 '1190176',
 '1190177',
 '1190178',
 '1190179',
 '1190180',
 '1190181',
 '1190182',
 '1190183',
 '1190184',
 '1190185',
 '1190186',
 '1190187',
 '1190188',
 '1190189',
 '1190190',
 '1190191',
 '1190192',
 '1190193',
 '1190194',
 '1190195',
 '1190196',
 '1190197',
 '1190198',
 '1190199',
 '1190200',
 '1190201',
 '1190202',
 '1190203',
 '1190204',
 '1190205',
 '1190206',
 '1190207',
 '1190208',
 '1190209',
 '1190210',
 '1190211',
 '1190213',
 '1190214',
 '1190215',
 '1190216',
 '1190217',
 '1190218',
 '1190219',
 '1190220',
 '1190221',
 '1190222',
 '1190223',
 '1190224',
 '1190225',
 '1190226',
 '1190227',
 '1190228',
 '1190229',
 '1190230',
 '1190231',
 '1190232',
 '1190233',
 '1190234',
 '1190235',
 '1190236',
 '1190237',
 '1190238',
 '1190239',
 '1190240',
 '1190241',
 '1190242',
 '1190243',
 '1190244',
 '1190245',
 '1190246',
 '1190247',
 '1190248',
 '1190249',
 '1190250',
 '1190251',
 '1190252',
 '1190253',
 '1190254',
 '1190255',
 '1190256',
 '1190257',
 '11

In [112]:
class ImportMatch():
    
    def __init__(self,league,season,matchid,jsonpath=JSONPATH):
        
        self.league = league
        self.season = season
        self.leaguename = self.league.lower().replace(" ","_")
        self.matchid = matchid
        self.jsonpath = jsonpath
        self.matchpath = jsonpath+self.leaguename+"/"+self.season+"/"+str(self.matchid)+".json"
        self.match = json.load(open(self.matchpath,"r",encoding="utf-8"),encoding="utf-8")
        
        self.__matchInfo__()
        self.__events__()
        self.__players__()
        self.__teams__()
        
    def __matchInfo__(self):
        
        matches_dict = {}
        
        try:
            matches_dict["wsmatchid"] = self.matchid
        except KeyError:
            matches_dict["wsmatchid"] = np.nan
        try:
            matches_dict["league"] = self.league
        except KeyError:
            matches_dict["league"] = np.nan
        try:
            matches_dict["season"] = self.season
        except KeyError:
            matches_dict["season"] = np.nan
        try:
            matches_dict["date"] = pd.to_datetime(self.match["startTime"].replace("T"," "))
        except KeyError:
            matches_dict["date"] = np.nan
        try:
            matches_dict["hometeamid"] = self.match["home"]["teamId"]
        except KeyError:
            matches_dict["hometeamid"] = np.nan
        try:
            matches_dict["awayteamid"] = self.match["away"]["teamId"]
        except KeyError:
            matches_dict["awayteamid"] = np.nan
        try:
            matches_dict["hometeamname"] = self.match["home"]["name"]
        except KeyError:
            matches_dict["hometeamname"] = np.nan
        try:
            matches_dict["awayteamname"] = self.match["away"]["name"]
        except KeyError:
            matches_dict["awayteamname"] = np.nan
        try:
            matches_dict["homescore"] = self.match["score"].split(" : ")[0]
        except KeyError:
            matches_dict["homescore"] = np.nan
        try:
            matches_dict["awayscore"] = self.match["score"].split(" : ")[1]
        except KeyError:
            matches_dict["awayscore"] = np.nan
        try:
            if self.match["pkScore"] == '':
                matches_dict["homepkscore"] = np.nan
                matches_dict["awaypkscore"] = np.nan
            else:
                matches_dict["homepkscore"] = self.match["pkScore"].split(" : ")[0]
                matches_dict["awaypkscore"] = self.match["pkScore"].split(" : ")[1]
        except KeyError:
            matches_dict["homepkscore"] = np.nan
            matches_dict["awaypkscore"] = np.nan
        try:
            matches_dict["referee"] = self.match["referee"]["name"]
        except KeyError:
            matches_dict["referee"] = np.nan
        try:
            matches_dict["managerhome"] = self.match["home"]["managerName"]
        except KeyError:
            matches_dict["managerhome"] = np.nan
        try:
            matches_dict["manageraway"] = self.match["away"]["managerName"]
        except KeyError:
            matches_dict["manageraway"] = np.nan
        try:
            matches_dict["attendance"] = self.match["attendance"]
        except KeyError:
            matches_dict["attendance"] = np.nan
        try:
            matches_dict["venuename"] = self.match["venueName"]
        except KeyError:
            matches_dict["venuename"] = np.nan
        
        column_order = ["wsmatchid","league","season","date","hometeamid",
                        "awayteamid","hometeamname","awayteamname","homescore",
                        "awayscore","homepkscore","awaypkscore","referee",
                        "managerhome","manageraway","attendance","venuename"]
        
        self.matchinfo = pd.Series(matches_dict)[column_order]
        
    
    def __events__(self):
            
        events = self.match["events"]
        event_list=[]
        qual_list=[]

        for event in events:
            events_dict = {}
            try:
                events_dict["keyid"] = str(self.matchid)+str(event["id"])
            except KeyError:
                events_dict["keyid"] = np.nan
            try:
                events_dict["wsmatchid"] = self.matchid
            except KeyError:
                events_dict["wsmatchid"] = np.nan
            try:
                events_dict["wseventid"] = str(event["id"])
            except KeyError:
                events_dict["wseventid"] = np.nan
            try:
                events_dict["matcheventid"] = event["eventId"]
            except KeyError:
                events_dict["matcheventid"] = np.nan
            try:
                events_dict["minute"] = event["minute"]
            except KeyError:
                events_dict["minute"] = np.nan
            try:
                events_dict["second"] = event["second"]
            except KeyError:
                events_dict["second"] = np.nan
            try:
                events_dict["expandedminute"] = event["expandedMinute"]
            except KeyError:
                events_dict["expandedminute"] = np.nan
            try:
                events_dict["teamid"] = event["teamId"]
            except KeyError:
                events_dict["teamid"] = np.nan
            try:
                events_dict["playerid"] = event["playerId"]
            except KeyError:
                events_dict["playerid"] = np.nan
            try:
                events_dict["period"] = event["period"]["displayName"]
            except KeyError:
                events_dict["period"] = np.nan
            try:
                events_dict["typeid"] = event["type"]["value"]
            except KeyError:
                events_dict["typeid"] = np.nan
            try:
                events_dict["type"] = event["type"]["displayName"]
            except KeyError:
                events_dict["type"] = np.nan
            try:
                events_dict["outcometype"] = event["outcomeType"]["displayName"]
            except KeyError:
                events_dict["outcometype"] = np.nan
            try:
                events_dict["x"] = event["x"]
            except KeyError:
                events_dict["x"] = np.nan
            try:
                events_dict["y"] = event["y"]
            except KeyError:
                events_dict["y"] = np.nan

            event_list.append(events_dict)

            qualifiers = event["qualifiers"]
            for qualifier in qualifiers:
                    qual_dict = {}
                    try:
                        qual_dict["keyid"] = events_dict["keyid"]
                    except KeyError:
                        qual_dict["keyid"] = np.nan
                    try:
                        qual_dict["wsmatchid"] = self.matchid
                    except KeyError:
                        qual_dict["wsmatchid"] = np.nan
                    try:
                        qual_dict["wseventid"] = events_dict["wseventid"]
                    except KeyError:
                        qual_dict["wseventid"] = np.nan
                    try:
                        qual_dict["matcheventid"] = events_dict["matcheventid"]
                    except KeyError:
                        qual_dict["matcheventid"] = np.nan
                    try:
                        qual_dict["qualid"] = qualifier["type"]["value"]
                    except KeyError:
                        qual_dict["qualid"] = np.nan
                    try:
                        qual_dict["qualname"] = qualifier["type"]["displayName"]
                    except KeyError:
                        qual_dict["qualname"] = np.nan
                    try:
                        qual_dict["qualvalue"] = qualifier["value"]
                    except KeyError:
                        qual_dict["qualvalue"] = 1
                    qual_list.append(qual_dict)
    
        self.events = pd.DataFrame(event_list)[["keyid","wsmatchid","wseventid","matcheventid","minute","second","expandedminute","teamid",
                                   "playerid","period","typeid","type","outcometype","x","y"]]
        
        self.qualifiers = pd.DataFrame(qual_list)[["keyid","wsmatchid","wseventid","matcheventid","qualid","qualname","qualvalue"]]
    
    def __players__(self):
        
        dic = [{"playerid":float(k),"playername":v} for k,v in self.match["playerIdNameDictionary"].items()]
        self.players = pd.DataFrame(dic,index=range(len(dic)))
        
    def __teams__(self):
        
        d = {"home":{"teamid":self.matchinfo["hometeamid"],
                "teamname":self.matchinfo["hometeamname"]},
         
        "away":{"teamid":self.matchinfo["awayteamid"],
                "teamname":self.matchinfo["awayteamname"]}}
        
        self.teams = pd.DataFrame(d).T
        

In [113]:
b = ImportMatch("Premier League","2017-2018",1190191)



In [114]:
b.teams

teamid teamname
away     13  Arsenal
home     15  Chelsea

In [18]:
a = b.events.merge(b.qualifiers,on = ["keyid","wsmatchid","wseventid","matcheventid"],how="left")\
.merge(b.players, on="playerid",how="left").merge(b.teams,on="teamid",how="left").merge(pd.DataFrame(b.matchinfo).T,on="wsmatchid",how="left")

In [53]:
q = b.events["type"] == "FormationSet"

b.events[q]["keyid"].tolist()

['11901911126208148.0', '11901912055767054.0']

In [49]:
np.unique(b.events["type"])

array(['Aerial', 'BallRecovery', 'BallTouch', 'BlockedPass', 'Card',
       'Challenge', 'Claim', 'Clearance', 'CornerAwarded', 'Dispossessed',
       'End', 'Error', 'FormationChange', 'FormationSet', 'Foul',
       'Interception', 'KeeperPickup', 'MissedShots', 'OffsideGiven',
       'OffsidePass', 'OffsideProvoked', 'Pass', 'Save', 'SavedShot',
       'ShieldBallOpp', 'ShotOnPost', 'Start', 'SubstitutionOff',
       'SubstitutionOn', 'Tackle', 'TakeOn'], dtype=object)

In [71]:
ids = b.qualifiers[b.qualifiers["keyid"].isin(['11901911126208148.0', '11901912055767054.0'])].qualvalue.values[10].split(",")

In [95]:
ai = list(map(int,ids[:11]))

b.players[b.players["playerid"].isin(ai)].set_index("playerid")

playername
playerid                     
110260.0       Sead Kolasinac
6775.0              Petr Cech
73078.0   Alexandre Lacazette
30051.0     Laurent Koscielny
80921.0      Shkodran Mustafi
136824.0           Alex Iwobi
23072.0         Nacho Monreal
125211.0      Héctor Bellerín
89401.0          Granit Xhaka
39308.0         Danny Welbeck
26820.0          Aaron Ramsey

In [99]:
{b.players[b.players["playerid"] == ai[0]]["playername"].values}

array(['Petr Cech'], dtype=object)

In [77]:
b.qualifiers[b.qualifiers["keyid"].isin(['11901911126208148.0', '11901912055767054.0'])]

keyid  wsmatchid     wseventid  matcheventid  qualid  \
7861  11901911126208148.0    1190191  1126208148.0             1     130   
7862  11901911126208148.0    1190191  1126208148.0             1     194   
7863  11901911126208148.0    1190191  1126208148.0             1      59   
7864  11901911126208148.0    1190191  1126208148.0             1      30   
7865  11901911126208148.0    1190191  1126208148.0             1      44   
7866  11901911126208148.0    1190191  1126208148.0             1     131   
7867  11901912055767054.0    1190191  2055767054.0             1      44   
7868  11901912055767054.0    1190191  2055767054.0             1     194   
7869  11901912055767054.0    1190191  2055767054.0             1     130   
7870  11901912055767054.0    1190191  2055767054.0             1     131   
7871  11901912055767054.0    1190191  2055767054.0             1      30   
7872  11901912055767054.0    1190191  2055767054.0             1      59   

                 qualname                                          qualvalue  
7861        TeamFormation                                                 17  
7862      CaptainPlayerId                                              11981  
7863         JerseyNumber  13, 15, 3, 24, 30, 28, 7, 4, 9, 22, 11, 1, 2, ...  
7864      InvolvedPlayers  73798,33064,84008,11981,27586,25931,114075,804...  
7865       PlayerPosition  1, 3, 3, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5...  
7866  TeamPlayerFormation  1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0,...  
7867       PlayerPosition  1, 3, 3, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 5, 5...  
7868      CaptainPlayerId                                              30051  
7869        TeamFormation                                                 17  
7870  TeamPlayerFormation  1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 0, 0, 0,...  
7871      InvolvedPlayers  6775,125211,110260,23072,80921,30051,26820,894...  
7872         JerseyNumber  33, 24, 31, 18, 20, 6, 8, 29, 9, 17, 23, 4, 7,...

In [32]:
f = open(files[0],"r",encoding="utf-8")
match = json.load(f,encoding="utf-8")

In [111]:
b.qualifiers[b.qualifiers["keyid"].isin(["11901911470815451.0","11901911906300230.0"])]

keyid  wsmatchid     wseventid  matcheventid  qualid  \
0  11901911470815451.0    1190191  1470815451.0             3     141   
1  11901911470815451.0    1190191  1470815451.0             3      56   
2  11901911470815451.0    1190191  1470815451.0             3     140   
3  11901911470815451.0    1190191  1470815451.0             3     212   
4  11901911470815451.0    1190191  1470815451.0             3     213   
5  11901911906300230.0    1190191  1906300230.0             4     212   
6  11901911906300230.0    1190191  1906300230.0             4     140   
7  11901911906300230.0    1190191  1906300230.0             4      56   
8  11901911906300230.0    1190191  1906300230.0             4     141   
9  11901911906300230.0    1190191  1906300230.0             4     213   

   qualname qualvalue  
0  PassEndY      47.9  
1      Zone    Center  
2  PassEndX      52.5  
3    Length       3.0  
4     Angle       5.8  
5    Length      28.4  
6  PassEndX      57.5  
7      Zone     Right  
8  PassEndY       2.0  
9     Angle       4.8

In [33]:
match.keys()

dict_keys(['playerIdNameDictionary', 'periodMinuteLimits', 'timeStamp', 'attendance', 'venueName', 'referee', 'weatherCode', 'elapsed', 'startTime', 'startDate', 'score', 'htScore', 'ftScore', 'etScore', 'pkScore', 'statusCode', 'periodCode', 'home', 'away', 'maxMinute', 'minuteExpanded', 'maxPeriod', 'expandedMinutes', 'expandedMaxMinute', 'periodEndMinutes', 'commonEvents', 'events', 'timeoutInSeconds'])

In [35]:
pd.Series(match['playerIdNameDictionary'])
#pd.Series(match["playerIdNameDictionary"])

104749               Riyad Mahrez
106981                Jamie Vardy
108055              Islam Slimani
110260             Sead Kolasinac
12431             Christian Fuchs
125209             Mohamed Elneny
125211            Héctor Bellerín
136824                 Alex Iwobi
136945               Demarai Gray
13756                  Mesut Özil
13796                Theo Walcott
19545           Kasper Schmeichel
23072               Nacho Monreal
23683               Danny Simpson
243552             Daniel Amartey
24444              Olivier Giroud
26222              Shinji Okazaki
26820                Aaron Ramsey
288795                Rob Holding
289253          Kelechi Iheanacho
29832                   Ben Hamer
299272               Ben Chilwell
32018                   Andy King
327683              Wilfred Ndidi
36745                David Ospina
39308               Danny Welbeck
42147             Marc Albrighton
6775                    Petr Cech
69738            Francis Coquelin
73078         

In [20]:
match["home"].keys()

dict_keys(['teamId', 'formations', 'stats', 'incidentEvents', 'shotZones', 'name', 'countryName', 'players', 'managerName', 'scores', 'field', 'averageAge'])

In [44]:
test = pd.DataFrame(match["events"])
    

In [45]:
test[sorted(test.columns)]

blockedX  blockedY                                cardType  endX  endY  \
0          NaN       NaN                                     NaN   NaN   NaN   
1          NaN       NaN                                     NaN   NaN   NaN   
2          NaN       NaN                                     NaN  28.8  30.1   
3          NaN       NaN                                     NaN  52.3  21.5   
4          NaN       NaN                                     NaN  44.3  20.7   
5          NaN       NaN                                     NaN  50.3   4.2   
6          NaN       NaN                                     NaN  70.5   5.0   
7          NaN       NaN                                     NaN  75.4  21.6   
8          NaN       NaN                                     NaN  34.6  85.1   
9          NaN       NaN                                     NaN   NaN   NaN   
10         NaN       NaN                                     NaN   NaN   NaN   
11         NaN       NaN                                     NaN  59.8  36.4   
12         NaN       NaN                                     NaN  46.0  56.6   
13         NaN       NaN                                     NaN  70.1  51.2   
14         NaN       NaN                                     NaN   NaN   NaN   
15         NaN       NaN                                     NaN   NaN   NaN   
16         NaN       NaN                                     NaN  48.8  58.5   
17         NaN       NaN                                     NaN  74.4  42.2   
18         NaN       NaN                                     NaN  38.8  76.1   
19         NaN       NaN                                     NaN  30.9  68.7   
20         NaN       NaN                                     NaN   NaN   NaN   
21         NaN       NaN                                     NaN  31.5  90.3   
22         NaN       NaN                                     NaN  25.0  95.3   
23         NaN       NaN                                     NaN  65.0  92.7   
24         NaN       NaN                                     NaN  39.9  13.4   
25         NaN       NaN                                     NaN  69.4  82.9   
26         NaN       NaN                                     NaN  69.9  92.5   
27         NaN       NaN                                     NaN  81.0  70.0   
28         NaN       NaN                                     NaN   NaN   NaN   
29         NaN       NaN                                     NaN   4.7  47.8   
...        ...       ...                                     ...   ...   ...   
1591       NaN       NaN                                     NaN   NaN   NaN   
1592       NaN       NaN                                     NaN   NaN   NaN   
1593       NaN       NaN                                     NaN   NaN   NaN   
1594       NaN       NaN                                     NaN   NaN   NaN   
1595       NaN       NaN                                     NaN   NaN   NaN   
1596       NaN       NaN                                     NaN  88.0  96.3   
1597       NaN       NaN                                     NaN  73.1  54.6   
1598       NaN       NaN                                     NaN  86.6  42.7   
1599       NaN       NaN                                     NaN  30.8  88.4   
1600       NaN       NaN                                     NaN   NaN   NaN   
1601       NaN       NaN                                     NaN  44.0  86.8   
1602       NaN       NaN                                     NaN   NaN   NaN   
1603       NaN       NaN                                     NaN   NaN   NaN   
1604       NaN       NaN  {'value': 31, 'displayName': 'Yellow'}   NaN   NaN   
1605       NaN       NaN                                     NaN  80.0  71.1   
1606       NaN       NaN                                     NaN  26.0  27.1   
1607       NaN       NaN                                     NaN   NaN   NaN   
1608       NaN       NaN                                     NaN  27.8  27.1   
1609  

In [85]:
test.loc[275:287]

blockedX  blockedY cardType  endX  endY  eventId  expandedMinute  \
275       NaN       NaN      NaN  94.0   2.5    182.0              15   
276       NaN       NaN      NaN   NaN   NaN    183.0              15   
277       NaN       NaN      NaN   NaN   NaN    127.0              15   
278       NaN       NaN      NaN   NaN   NaN    184.0              15   
279       NaN       NaN      NaN   NaN   NaN      NaN              16   
280       NaN       NaN      NaN  59.5  85.0    129.0              16   
281       NaN       NaN      NaN  56.8  80.8    130.0              16   
282       NaN       NaN      NaN  79.5  80.9    131.0              16   
283       NaN       NaN      NaN   NaN   NaN    186.0              16   
284       NaN       NaN      NaN  30.2  30.7    187.0              16   
285       NaN       NaN      NaN  37.8   4.0    189.0              16   
286       NaN       NaN      NaN  27.7  13.2    190.0              16   
287       NaN       NaN      NaN  19.0  48.4    191.0              16   

     goalMouthY  goalMouthZ            id  ...   playerId  \
275         NaN         NaN  1.987124e+09  ...    89401.0   
276         NaN         NaN  1.153357e+09  ...   125211.0   
277         NaN         NaN  1.730164e+09  ...    42147.0   
278         NaN         NaN  1.778224e+09  ...   125211.0   
279         NaN         NaN  2.035997e+09  ...   106981.0   
280         NaN         NaN  2.057250e+09  ...    19545.0   
281         NaN         NaN  1.693701e+09  ...    26222.0   
282         NaN         NaN  2.035997e+09  ...   327683.0   
283         NaN         NaN  2.056132e+09  ...    23072.0   
284         NaN         NaN  1.407872e+09  ...   288795.0   
285         NaN         NaN  1.712080e+09  ...   125209.0   
286         NaN         NaN  1.887222e+09  ...   125211.0   
287         NaN         NaN  1.235486e+09  ...   288795.0   

                                            qualifiers  relatedEventId  \
275  [{'type': {'value': 213, 'displayName': 'Angle...             NaN   
276  [{'type': {'value': 233, 'displayName': 'Oppos...             NaN   
277  [{'type': {'value': 285, 'displayName': 'Defen...             NaN   
278  [{'type': {'value': 56, 'displayName': 'Zone'}...             NaN   
279                                                 []             NaN   
280  [{'type': {'value': 212, 'displayName': 'Lengt...             NaN   
281  [{'type': {'value': 212, 'displayName': 'Lengt...             NaN   
282  [{'type': {'value': 140, 'displayName': 'PassE...             NaN   
283                                                 []             NaN   
284  [{'type': {'value': 141, 'displayName': 'PassE...             NaN   
285  [{'type': {'value': 140, 'displayName': 'PassE...             NaN   
286  [{'type': {'value': 140, 'displayName': 'PassE...             NaN   
287  [{'type': {'value': 141, 'displayName': 'PassE...             NaN   

     relatedPlayerId                             satisfiedEventsTypes second  \
275              NaN  [90, 118, 116, 123, 126, 204, 35, 37, 216, 217]   51.0   
276              NaN                                         [90, 69]   56.0   
277              NaN                                        [90, 141]   56.0   
278              NaN                                             [90]   57.0   
279              NaN                                             [60]    NaN   
280              NaN            [90, 118, 116, 126, 35, 36, 215, 217]   24.0   
281              NaN             [90, 118, 116, 29, 34, 37, 215, 217]   28.0   
282              NaN                                        [90, 216]   33.0   
283              NaN                                             [61]   33.0   
284              NaN             [90, 116, 29, 131, 35, 36, 214, 217]   48.0   
285              NaN                  [90, 116, 29, 35, 37, 215, 217]   54.0   
286              NaN                  [90, 116, 29, 34, 36, 214, 217]   56.0   
287              NaN                  [90, 116, 29

In [66]:
test[["id","teamId","eventId","period","minute","second","type"]]

id  teamId  eventId  \
0     1.985789e+09      14      2.0   
1     1.969703e+09      13      3.0   
2     2.073174e+09      13      4.0   
3     1.757172e+09      13      5.0   
4     1.222799e+09      13      6.0   
5     1.753357e+09      13      7.0   
6     1.534782e+09      13      8.0   
7     1.540986e+09      13      9.0   
8     1.149035e+09      14      3.0   
9     1.663640e+09      14      4.0   
10    1.424479e+09      13     10.0   
11    1.178309e+09      13     11.0   
12    2.051165e+09      13     12.0   
13    1.090148e+09      13     13.0   
14    1.342435e+09      13     14.0   
15    1.991814e+09      14      5.0   
16    1.931687e+09      14      6.0   
17    1.634874e+09      13     15.0   
18    1.680886e+09      14      7.0   
19    1.439158e+09      14      8.0   
20    1.597736e+09      14      9.0   
21    1.622390e+09      14     10.0   
22    1.756221e+09      14     11.0   
23    1.960100e+09      14     12.0   
24    2.054420e+09      13     16.0   
25    1.487552e+09      14     13.0   
26    1.785098e+09      14     29.0   
27    1.267464e+09      14     14.0   
28    1.652710e+09      13     18.0   
29    1.880389e+09      13     19.0   
...            ...     ...      ...   
1591  1.160395e+09      13   1084.0   
1592  1.803612e+09      14    676.0   
1593  2.060238e+09      13   1086.0   
1594  1.512902e+09      13   1085.0   
1595  1.530263e+09      14    677.0   
1596  1.724735e+09      13   1087.0   
1597  1.387857e+09      14    678.0   
1598  1.249387e+09      14    679.0   
1599  2.145193e+09      13   1088.0   
1600  1.576198e+09      14    680.0   
1601  2.019854e+09      13   1090.0   
1602  1.211922e+09      14    682.0   
1603  1.368021e+09      13   1091.0   
1604  1.899309e+09      14    683.0   
1605  1.182441e+09      13   1092.0   
1606  1.987194e+09      14    685.0   
1607  1.975880e+09      14    686.0   
1608  1.509075e+09      14    687.0   
1609  1.592724e+09      13   1093.0   
1610  1.685648e+09      14    688.0   
1611  1.750335e+09      14    689.0   
1612  1.210643e+09      13   1094.0   
1613  2.037799e+09      13   1095.0   
1614  1.997425e+09      13   1096.0   
1615  1.382438e+09      13   1098.0   
1616  1.647164e+09      14    692.0   
1617  1.197891e+09      13   1100.0   
1618  1.883948e+09      14    693.0   
1619  1.765837e+09      13      1.0   
1620  2.132242e+09      14      1.0   

                                         period  minute  second  \
0      {'value': 1, 'displayName': 'FirstHalf'}       0     0.0   
1      {'value': 1, 'displayName': 'FirstHalf'}       0     0.0   
2      {'value': 1, 'displayName': 'FirstHalf'}       0     1.0   
3      {'value': 1, 'displayName': 'FirstHalf'}       0     2.0   
4      {'value': 1, 'displayName': 'FirstHalf'}       0     5.0   
5      {'value': 1, 'displayName': 'FirstHalf'}       0     7.0   
6      {'value': 1, 'displayName': 'FirstHalf'}       0     9.0   
7      {'value': 1, 'displayName': 'FirstHalf'}       0    11.0   
8      {'value': 1, 'displayName': 'FirstHalf'}       0    13.0   
9      {'value': 1, 'displayName': 'FirstHalf'}       0    14.0   
10     {'value': 1, 'displayName': 'FirstHalf'}       0    14.0   
11     {'value': 1, 'displayName': 'FirstHalf'}       0    15.0   
12     {'value': 1, 'displayName': 'FirstHalf'}       0    16.0   
13     {'value': 1, 'displayName': 'FirstHalf'}       0    19.0   
14     {'value': 1, 'displayName': 'FirstHalf'}       0    21.0   
15     {'value': 1, 'displayName': 'FirstHalf'}       0    21.0   
16     {'value': 1, 'displayName': 'FirstHalf'}       0    22.0   
17     {'value': 1, 'displayName': 'FirstHalf'}       0    24.0   
18     {'value': 1, 'displayName': 'FirstHalf'}       0    25.0   
19     {'value': 1, 'displayName': 'FirstHalf'}       0    27.0   
20     {'value': 1, 'displayName': 'FirstHalf'}       0    28.0   
21     {'value': 1, 'displayName': 'FirstHalf'}       0    29.0   
22     {'value': 1, 'displayName': 'FirstHalf'}       0 

In [79]:
pd.DataFrame(test["qualifiers"]).loc[3].values

array([ list([{'type': {'value': 155, 'displayName': 'Chipped'}}, {'type': {'value': 213, 'displayName': 'Angle'}, 'value': '6.1'}, {'type': {'value': 56, 'displayName': 'Zone'}, 'value': 'Center'}, {'type': {'value': 140, 'displayName': 'PassEndX'}, 'value': '52.3'}, {'type': {'value': 212, 'displayName': 'Length'}, 'value': '24.0'}, {'type': {'value': 141, 'displayName': 'PassEndY'}, 'value': '21.5'}])], dtype=object)

In [86]:
test.sort_values(["minute","second"])

blockedX  blockedY                                cardType  endX  endY  \
0          NaN       NaN                                     NaN   NaN   NaN   
1          NaN       NaN                                     NaN   NaN   NaN   
1617       NaN       NaN                                     NaN   NaN   NaN   
1618       NaN       NaN                                     NaN   NaN   NaN   
1619       NaN       NaN                                     NaN   NaN   NaN   
1620       NaN       NaN                                     NaN   NaN   NaN   
2          NaN       NaN                                     NaN  28.8  30.1   
3          NaN       NaN                                     NaN  52.3  21.5   
4          NaN       NaN                                     NaN  44.3  20.7   
5          NaN       NaN                                     NaN  50.3   4.2   
6          NaN       NaN                                     NaN  70.5   5.0   
7          NaN       NaN                                     NaN  75.4  21.6   
8          NaN       NaN                                     NaN  34.6  85.1   
9          NaN       NaN                                     NaN   NaN   NaN   
10         NaN       NaN                                     NaN   NaN   NaN   
11         NaN       NaN                                     NaN  59.8  36.4   
12         NaN       NaN                                     NaN  46.0  56.6   
13         NaN       NaN                                     NaN  70.1  51.2   
14         NaN       NaN                                     NaN   NaN   NaN   
15         NaN       NaN                                     NaN   NaN   NaN   
16         NaN       NaN                                     NaN  48.8  58.5   
17         NaN       NaN                                     NaN  74.4  42.2   
18         NaN       NaN                                     NaN  38.8  76.1   
19         NaN       NaN                                     NaN  30.9  68.7   
20         NaN       NaN                                     NaN   NaN   NaN   
21         NaN       NaN                                     NaN  31.5  90.3   
22         NaN       NaN                                     NaN  25.0  95.3   
23         NaN       NaN                                     NaN  65.0  92.7   
24         NaN       NaN                                     NaN  39.9  13.4   
25         NaN       NaN                                     NaN  69.4  82.9   
...        ...       ...                                     ...   ...   ...   
1588       NaN       NaN                                     NaN  60.2   4.0   
1589       NaN       NaN                                     NaN  67.5  13.2   
1590       NaN       NaN                                     NaN   NaN   NaN   
1591       NaN       NaN                                     NaN   NaN   NaN   
1592       NaN       NaN                                     NaN   NaN   NaN   
1593       NaN       NaN                                     NaN   NaN   NaN   
1594       NaN       NaN                                     NaN   NaN   NaN   
1595       NaN       NaN                                     NaN   NaN   NaN   
1596       NaN       NaN                                     NaN  88.0  96.3   
1574       NaN       NaN                                     NaN   NaN   NaN   
1597       NaN       NaN                                     NaN  73.1  54.6   
1598       NaN       NaN                                     NaN  86.6  42.7   
1599       NaN       NaN                                     NaN  30.8  88.4   
1600       NaN       NaN                                     NaN   NaN   NaN   
1601       NaN       NaN                                     NaN  44.0  86.8   
1602       NaN       NaN                                     NaN   NaN   NaN   
1603       NaN       NaN                                     NaN   NaN   NaN   
1604       NaN       NaN  {'value': 31, 'displayName': 'Yellow'}   NaN   NaN   
1605  

In [133]:
np.fv(pmt=-150000,nper=40*12,pv=0,rate=0.05/12)

228903023.46311301

In [124]:
2000000*0.1

200000.0

In [126]:
1900000000/12

158333333.33333334

In [128]:
32000*0.1/12

266.6666666666667

In [129]:
266*750

199500